<a href="https://colab.research.google.com/github/MatiasManchino/curso_python2022/blob/main/ScrapBeer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# -------------------------------------------------------IMPORTANTE---------------------------------------------------------------


La pagina scrapiada esta estructurada en dos versiones la del 2015 y la del 2021, en el programa solo tomo la del 2021, con exepcion de las categorias M y C las cuales no tienen esa version. Esta separado a su vez por categorias de cervezas y en cada una hay 1 o mas cervezas, las cuales tienen sus caracteristicas determinadas, estas por desgracia no siempre son las mismas,aparecen:

**impression, appearance, aroma, flavor, mouthfeel, comments, history, ingredients, instructions, style-comparison, visual-statistics, commercial-explence, style-attributes**

 La caracteristica "ingredients", al aprecer deja de aparecer y aparece "instructions", la cual por ahí a futuro se podrían unir las columnas. 
"visual-statistics" es otra tabla en la mayoria de los casos esta estructurada en el HTML con un "div" de clase "table", TODAVÍA NO ME DECIDI COMO ORDENAR ESOS DATOS.

 Y existe un inveniente que es caotico para el metodo de scraping utilizado, el cual consiste en que existen parrafos entre la primera carateristica y el titulo de algunas cervezas que TODAVÍA NO ENCUENTRO LA MANERA DE EXTRAERLAS.

FALTA TRADUCIR

In [ ]:
from bs4 import BeautifulSoup
from IPython.display import HTML
import pandas as pd #para crear la bbdd
import numpy as np #para completar los valores nulos
import requests
import re

url = 'http://httpbin.org/headers' 
nuestros_headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
    }
resp_con_headers = requests.get(url, headers = nuestros_headers)



enlaces_de_cada_birra = [] #la lista para meter todos los enlaces de las categorias de las cervezas en una lista

for n in range(1,35): #cargo en la lista de enlaces de distitas categorias de birras del 1 al 34

    #en la pagina hay dos versiones la del 2015 y la del 2021, tome la ultima en este caso
    enlaces_de_cada_birra.append(f"https://www.bjcp.org/style/2021/{n}")


for n in range(1,4): #cargo en la lista de enlaces de distitas categorias de birras del M1 al M3
    
    enlaces_de_cada_birra.append(f"https://www.bjcp.org/style/2015/M{n}") 

for n in range(1,3): #cargo en la lista de enlaces de distitas categorias de birras del C1 al C2
    
    enlaces_de_cada_birra.append(f"https://www.bjcp.org/style/2015/C{n}")

In [ ]:
#una funcion que no llegue a resolver como aplicarla pero seria lo ideal, para mejorar la modularidad del programa
def encapsular_variables ():

    categoria = []
    nombre = []
    impression = []
    appearance = []
    aroma = []
    flavor = []
    mouthfeel = []
    comments = []
    history = []
    ingredients = [] 
    instructions = []
    comparison = []
    statistics = []
    commercial_exp = []
    style_attributes = []
    variables = dir()

In [ ]:
#una funcion que no llegue a resolver como aplicarla pero seria lo ideal, para mejorar la modularidad del programa
def agregar_a_lista(texto, clase, subindice):
    #pasa el nombre de la variable a un valor para que se pueda comparar
    print(clase)
    
    texto.append(subindice.find('div', class_ =f'{clase}').text.strip())
    print(texto)

In [ ]:
#se puede mejorar la eficiencia, modularizando
def cada_pag(enl):

    #para guardar el titulo y descripcion de la categoria entera
    #cada una con su identificador en el titulo (sin aislar del resto del string)
    titulo_cat = []
    descrip_cat = []

    #para guardar las caracteristicas de cada cerveza
    #lleva con sigo un indice llamado "categoria" para poder identificar la categoria
    categoria = []
    nombre = []
    impression = []
    appearance = []
    aroma = []
    flavor = []
    mouthfeel = []
    comments = []
    history = []
    ingredients = [] 
    instructions = []
    comparison = []
    statistics = []
    commercial_exp = []
    style_attributes = []
    #variables = dir() #esto se usaria si modularizo la funcion

    for link in enl: #iterdo

        resultado = requests.get(link) #extrae los datos de la url
        codigo_html = resultado.text #convierte a texto el codigo HTML
        soup = BeautifulSoup(codigo_html, "html.parser") #parsea el texto: 
                                            #estructurarlo para que sea legible

        articulo = soup.find_all('article') #extrae todos tag que sea "article"

        for subindice in articulo: #itero en los "article" que aparecen en la pag
        
            #se crea una regla de busqueda para distinguir la categoria
            #de las cervezas
            regla_de_busqueda = 'class="overall-impression"><h2>'
            condicion = [m for m in re.findall(regla_de_busqueda, str(subindice))]

            #-------- Titulo y descripcion de la categoria

            if condicion == []:
                #guardo en las caracteristicasde la birra o de la categoria
                #que aparece en los distintos TAG que hay en la pagina
                titulo_cat.append(articulo[0].find('h1').text.strip())

                descrip_cat.append(articulo[0].find('p').text.strip())
            else:
                #-------- Dentro de las birras generales

                #guardo los dos digitos de la categoria (del 1 al 9 hay que limpar el dato)
                categoria.append(link[-2]+link[-1]) 

                nombre.append(subindice.find('a').text.strip())
                
                impression.append(subindice.find('div', class_ =
                                                 "overall-impression").text.strip())
                
                #en algunas aparece y en otras no, en las que no, le agrego "No tiene"
                try: 
                    appearance.append(subindice.find('div', class_ =
                                                     "appearance").text.strip())

                except:
                    if categoria == 34: #la pagina 34 une todas
                        appearance.append(subindice.find('div', class_ =
                                                        "aroma").text.strip())
                        aroma.append(subindice.find('div', class_ =
                                                    "aroma").text.strip())
                        flavor.append(subindice.find('div', class_ =
                                                    "aroma").text.strip())
                        mouthfeel.append(subindice.find('div', class_ =
                                                        "aroma").text.strip())

                try:
                    aroma.append(subindice.find('div', class_ ="aroma").text.strip())

                except: 
                    if categoria != 34: #la pagina 34 une todas
                        aroma.append(["No tiene"])


                try:
                    flavor.append(subindice.find('div', class_ ="flavor").text.strip())

                except: 
                    if categoria != 34: #la pagina 34 une todas
                        flavor.append(["No tiene"])

                
                try:
                    mouthfeel.append(subindice.find('div', class_ =
                                                    "mouthfeel").text.strip())

                except: 
                    if categoria != 34: #la pagina 34 une todas
                        mouthfeel.append(["No tiene"])


                try: #la pagina M1 no tiene
                    comments.append(subindice.find('div', class_ =
                                                   "comments").text.strip())

                except:
                    comments.append(["No tiene"])


                try: #la 2da de la pagina 21 no tiene
                    history.append(subindice.find('div', class_ ="history").text.strip())

                except:
                    history.append(["No tiene"])


                try: #la 2da de la pagina 21 no tiene
                    ingredients.append(subindice.find('div', class_ =
                                                      "ingredients").text.strip())

                except:
                    ingredients.append(["No tiene"])


                try: #la 2da de la pagina 21 no tiene
                    instructions.append(subindice.find('div', class_ =
                                                       "ingredients").text.strip())

                except:
                    instructions.append(["No tiene"])


                try: #la 2da de la pagina 29 no tiene
                    comparison.append(subindice.find('div', class_ =
                                                     "style-comparison").text.strip())

                except:
                    comparison.append(["No tiene"])


                try: #la 2da de la pagina 21 no tiene
                    commercial_exp.append(subindice.find('div', class_ =
                                                         "commercial-examples").text.strip())

                except:
                    commercial_exp.append(["No tiene"])


                try: #la 2da de la pagina 21 no tiene
                    style_attributes.append(subindice.find('div', class_ =
                                                           "style-attributes").text.strip())

                except:
                    style_attributes.append(["No tiene"])

                #for n in variables:
                #    globals()[n] = []
                #    variable = n
                #    agregar_a_lista(appearance,variable,subindice)

    #una vez llenado todas las listas, armo una lista con listas para retornar
    #y utilizarlas fuera de la funcion    
    lista_de_columnas = [categoria,nombre, impression, appearance, aroma, flavor,
                         mouthfeel, comments, history, ingredients, instructions,
                         comparison, statistics, commercial_exp, style_attributes]
    return lista_de_columnas
        
lista_de_columnas = cada_pag(enlaces_de_cada_birra)


In [ ]:
df = pd.DataFrame() #creo el dataframe

#creo las columnas pasando las listas
df['categoria'] =  lista_de_columnas[0]
df['nombre'] =  lista_de_columnas[1]
df['impresion general'] =  lista_de_columnas[2]
df['apariencia'] =  pd.Series(lista_de_columnas[3])
df['aroma'] =  lista_de_columnas[4]
df['sabor'] =  lista_de_columnas[5]
df['textura'] =  lista_de_columnas[6]
df['comentarios'] =  lista_de_columnas[7]
df['historia'] =  lista_de_columnas[8]
df['ingredientes'] =  lista_de_columnas[9]
df['instrucciones'] = pd.Series(lista_de_columnas[10])
df['comparacion'] =  pd.Series(lista_de_columnas[11])
df['estadisticas'] =  pd.Series(lista_de_columnas[12])
df['ejemplos comerciales'] =  pd.Series(lista_de_columnas[13])
df['tipos de atributos'] =  pd.Series(lista_de_columnas[14])

df.head()

In [ ]:
#pip install googletrans

In [ ]:
#from googletrans import Translator